In [1]:
from Crypto.Util.number import inverse, getPrime, long_to_bytes, bytes_to_long

# Prerequisites

- PRG
- modular arithmetic

# Theory

- https://en.wikipedia.org/wiki/Linear_congruential_generator
- https://brilliant.org/problems/breaking-linear-congruential-generators/
- https://math.stackexchange.com/questions/2724959/how-to-crack-a-linear-congruential-generator-lcg

A linear congruential generator (LCG) is an algorithm that yields a sequence of pseudo-randomized numbers calculated with a discontinuous piecewise linear equation.

It is defined by the following recurrence relation:
- $S_{m+1} = (aS_m + c) \bmod n$

where
- $n$ = the modulus
- $a$ = the multiplier
- $c$ = the increment
- $S_0$ = the seed (start value)


## An attack

Let Eve know 3 plaintexts $m_1, m_2, m_3$ for 3 ciphertexts $c_1, c_2, c_3$
- first she computes $S_1, S_2, S_3$
- Then she has
    - $S_2 \equiv a\cdot S_1 + c \bmod n$
    - $S_3 \equiv a\cdot S_3 + c \bmod n$
- Which is a system of equations => 
    - $a = (S_2 - S_3) (S_1 - S_2)^{-1} \bmod n$
    - $c = S_2 - S_1 (S_2 - S_3)(S_1 - S_2)^{-1} \bmod n$
- From now on we can compute everything

# Code

In [8]:
def lcg(modulus, a, c, seed):
    """Linear congruential generator."""
    while True:
        seed = (a * seed + c) % modulus
        yield seed

In [9]:
n = getPrime(512)
a = getPrime(256)
c = getPrime(256)
s0 = getPrime(256)
print(a, c, n)

110924620051623968447266268873007437161656047175321232942762078096800785291629 78330055002505665395054081034167722403903079478840676381279591427631187922487 7554986015269310430292520786725319659626669517486107306231817531558561772180102176996548849487708798671234713961543501834980727440531691912787827507581477


In [21]:
s_list = []
for i, s in enumerate(lcg(n, a, c, s0)):
    if i == 100:
        break
    s_list.append(s)
s_list = s_list[-3:]
print()

In [22]:
s_list

[7069843410600921750773216882817131747030272542317631706718102992809185998190651209087209896703063661290035343302422897997519948973036744711053617418084672,
 6381658174375721988221849622225151332002647602829904206260936732096794761994834683367137126578371140647084566044901718545192610854134126214359859505195708,
 1164514516946568060322739436593388053704035783550399779338228888731094291364275001726254939824356384434223109829070637246326191067862562083514228611484436]

In [23]:
a_decr = ((s_list[1] - s_list[2]) * inverse((s_list[0] - s_list[1]), n)) % n
c_decr = (s_list[1] - s_list[0] * (s_list[1] - s_list[2]) * inverse(s_list[0] - s_list[1], n)) % n

In [24]:
a_decr, c_decr

(110924620051623968447266268873007437161656047175321232942762078096800785291629,
 78330055002505665395054081034167722403903079478840676381279591427631187922487)

In [25]:
a_decr == a, c_decr == c

(True, True)

# Resources

https://www.youtube.com/watch?v=PtEivGPxwAI